# Interactive Audit Testing

This notebook allows for real-time testing of the IFRS9 Automated Auditor without using web ports.

In [1]:
# Cell 1: Load environment and initialize RcmAuditor
import os
from dotenv import load_dotenv
from rcm_engine import RcmAuditor
from config import CONFIG

load_dotenv()

# Initialize Auditor
auditor = RcmAuditor()
print("RcmAuditor initialized.")

# Ensure documents are indexed
auditor.initialize_rag()
print("RAG Index ready.")

RcmAuditor initialized.
Loading documents from documents/...
Loading Política de Previsionamiento_modificada sep 2025.pdf...
Loading Respuesta Memorando final Inspección 2025_Perdida Esperada NIIF.pdf...
Creating vector store with 58 chunks (Size: 1500, Overlap: 300)...
Processing batch 1/6 (10 chunks)...
Processing batch 2/6 (10 chunks)...
Processing batch 3/6 (10 chunks)...
Processing batch 4/6 (10 chunks)...
Processing batch 5/6 (10 chunks)...
Processing batch 6/6 (8 chunks)...
Index built successfully.
RAG Index ready.


In [4]:
from config import CONFIG

CONFIG

{'llm_settings': {'provider': 'google',
  'temperature': 0.0,
  'openai': {'model': 'gpt-4o-mini'},
  'google': {'model': 'models/gemini-2.0-flash'}},
 'rag_settings': {'chunk_size': 1500,
  'chunk_overlap': 300,
  'document_language': 'Spanish'},
 'paths': {'input_csv': 'rcm_input.csv',
  'output_json': 'audit_results.json',
  'documents_folder': 'documents/'},
 'validation': {'enable_self_critique': True}}

In [7]:
# Cell 1.5: Dynamic Configuration (LLM Provider & Language)
from config import CONFIG
from llm_factory import reload_config_and_reinit
import os

print(f"Current Provider: {CONFIG['llm_settings']['provider']}")
print(f"Current Language: {auditor.rag_engine.doc_language}")

# Function to switch provider dynamically
def switch_provider(new_provider):
    CONFIG['llm_settings']['provider'] = new_provider
    # Re-initialize Auditor with new config
    global auditor
    auditor = RcmAuditor() # This uses the factory which reads the updated CONFIG
    auditor.initialize_rag()
    print(f"Switched to provider: {new_provider}")

# Example: Uncomment to switch to Google
# switch_provider('google')
# switch_provider('openai')

Current Provider: google
Current Language: Spanish


In [8]:
# Cell 2: Define a specific test question manually
test_question = "Is lifetime PD estimation well explained and consistent with the expected life of exposures and staging approach?"#"Como se calcula la PD a 12 meses?"
print(f"Test Question: {test_question}")

Test Question: Is lifetime PD estimation well explained and consistent with the expected life of exposures and staging approach?


In [9]:
# Cell 3: Run the retrieval step only and print retrieved PDF text chunks
print("Retrieving top 5 chunks...")

# Verify HyDE generation
print(f"Generating HyDE query for: '{test_question}'")
hyde_query = auditor.rag_engine.generate_search_query(test_question)
print(f"HyDE Output ({auditor.rag_engine.doc_language}):\n{hyde_query}\n")

retrieved_docs = auditor.rag_engine.retrieve(test_question, k=5)

for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Chunk {i+1} (Page {doc.metadata.get('page', 'N/A')}) ---")
    print(doc.page_content[:500] + "...") # Print first 500 chars

Retrieving top 5 chunks...
Generating HyDE query for: 'Is lifetime PD estimation well explained and consistent with the expected life of exposures and staging approach?'


ChatGoogleGenerativeAIError: Error calling model 'models/gemini-2.0-flash' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\nPlease retry in 37.478545922s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas

In [ ]:
# Cell 4: Run the full process_row logic and print AI Answer AND Critique Score

# Mock a row data structure
mock_row = {
    'Control Reference': 'Test-Ref-001',
    'Test Procedure': test_question
}

print("Running full process_row...")
result = auditor.process_row(mock_row)

print("\n=== AI Answer ===")
print(result['AI_Answer'])

print("\n=== Validation ===")
print(f"Score: {result['Validation_Score']}")
print(f"Reasoning: {result['Validation_Reasoning']}")